In [ ]:
import sys
sys.path.append('../')

import pandas as pd
import cobra
import re

# Load heterologous reactions
# df = pd.read_csv('../results/C00024_C06142_metabolites_reactions.csv')
df = pd.read_csv('../results/C00084_C06142_metabolites_reactions.csv')

# Load GSMM
model = cobra.io.load_json_model('../data/original/e_coli_core.json')

# Impose anaerobic conditions
model.reactions.get_by_id('EX_o2_e').bounds = (0, 1000)

# Print number of reactions and metabolites in the model
print('Number of reactions:', len(model.reactions))
print('Number of metabolites:', len(model.metabolites))

In [ ]:
# Add new line to df using pd.concat
df = df.append({'Reaction': 'R_ex', 'Equation': 'C06142 --> C06142_e'}, ignore_index=True)
df = df.append({'Reaction': 'EX_1btol', 'Equation': 'C06142_e -->'}, ignore_index=True)

In [ ]:
# Add metabolites to the model
# Extract the metabolites from the equation
def get_metabolites_to_add_to_GEM(df: pd.DataFrame) -> list:
    mapping_table=str.maketrans({'-':'','+':'', '>':'','<':''})
    metabolites=[]
    for i in df.index:
        dummy= str(df.loc[i]['Equation'])
        dummy=dummy.translate(mapping_table)
        dummy=dummy.split()
        for e in dummy:
            if len(e)>2:
                metabolites.append(e)

    metabolites = list(set(metabolites))
    return metabolites

metabolites = get_metabolites_to_add_to_GEM(df)

# Add metabolites to the model
for m in metabolites:
    if m not in model.metabolites:
        model.add_metabolites([cobra.Metabolite(m, compartment='c')])

print('Number of metabolites:', len(model.metabolites))

for i in range(len(df)):
    reaction = cobra.Reaction(df['Reaction'].iloc[i])
    model.add_reactions([reaction])
    model.reactions.get_by_id(df['Reaction'].iloc[i]).reaction = df['Equation'].iloc[i]
    # model.reactions.get_by_id(df['Reaction'].iloc[i]).bounds = (-1000, 1000)

model.metabolites.get_by_id('C06142_e').compartment='e'     
# model.reactions.get_by_id('EX_1btol').bounds = (-1000, 0)

print('Number of reactions:', len(model.reactions))

for r in df['Reaction']:
    print(model.reactions.get_by_id(r))


In [ ]:
model.objective='EX_1btol'
solution=model.optimize()
the_yield="{:.0%}".format(solution.objective_value/abs(model.reactions.EX_glc__D_e.lower_bound))
print('The theoretical yield', the_yield)
print(solution)

In [ ]:
from cobra.flux_analysis import flux_variability_analysis

model.objective = model.reactions.get_by_id('BIOMASS_Ecoli_core_w_GAM')
# model.objective = model.reactions.get_by_id('EX_1btol')
flux_variability_analysis(model, fraction_of_optimum=0.8)